In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Definir la ruta del archivo
file_path = '/path/egwas_BrownSpolar.egwas'

# Leer el archivo por partes usando chunksize
chunksize = 100000  # ajustar este valor según tu memoria disponible
chunks = pd.read_csv(file_path, delim_whitespace=True, chunksize=chunksize)

# Procesar los datos por partes y almacenarlos en una lista
data_list = []

for chunk in chunks:
    # Reemplazar comas por puntos en las columnas numéricas y convertirlas a numéricas
    chunk['P'] = pd.to_numeric(chunk['P'].str.replace(',', '.'), errors='coerce')
    data_list.append(chunk)

# Combinar todas las partes en un solo DataFrame
data = pd.concat(data_list)

# Crear columna -log10(P)
data['-log10(P)'] = -np.log10(data['P'])

# Filtrar datos significativos
significant_threshold = 5e-8
data['significant'] = data['P'] < significant_threshold

# Crear el Manhattan plot
fig, ax1 = plt.subplots(figsize=(16, 4))

# Gráfico de todos los SNPs
ax1.scatter(data.index, data['-log10(P)'], c='black', s=7)

# Añadir la línea de significancia
ax1.axhline(y=-np.log10(significant_threshold), color='grey', linestyle='dotted')

# Añadir etiquetas y título
ax1.set_xlabel('SNP Index')
ax1.set_ylabel('-log10(P)')
plt.title('SNP outlier')

# Resaltar los SNPs más significativos
ax1.scatter(data[data['significant']].index, data[data['significant']]['-log10(P)'], c='red', s=7)

# Crear un segundo eje y para -ln(p-value)
ax2 = ax1.twinx()
ax2.set_ylabel('-ln(P)')
ax2.set_ylim(ax1.get_ylim())
ax2.set_yticks(ax1.get_yticks())
ax2.set_yticklabels([f"{-np.log(10**-y):.0f}" for y in ax1.get_yticks()])

plt.show()